<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# IPXact Example

## Motivation
Some explanation should go here

## Setup

In [ ]:
//unmanaged dep for ipxact
interp.load.cp(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(System.getProperty("user.dir") + "/../rocket-dsp-utils-assembly-1.0.jar")))

In [ ]:
import chisel3._
import chisel3.util._

---
# IPXact Background

A paragraph or two describing ipxact should go here

---
# FIR Filter (from 3.2)

For this module, we'll be using a slightly modified example from [Module 3.2: Generators: Collection](3.2_collections.ipynb).
If you haven't started Module 3.2, don't worry.
You'll learn about the details of how `MyManyDynamicElementVecFir` works, but the basic idea is that it is a FIR filter generator.

The generator has one parameter: length.
That parameter dictates how many taps the filter has.

The generator has 3 inputs:
* in, the input to the filter
* valid, a boolean that says when the input is valid
* consts, a vector for all the taps

and 1 output:
* out, the filtered input

<img src="images/fir.jpg" style="width:450px;"/>

In [ ]:
import chisel3._
import chisel3.util._

class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val valid = Input(Bool())
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => when (io.valid) { b := a } }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

---
# DspBlock

Integrating DSP components into a larger system can be challenging and error prone.
The [rocket-dsp-utils](https://github.com/ucb-art/rocket-dsp-utils) repository consists of useful generators that should help with such tasks.

One of the core abstractions is the notion of a DSPBlock.
A DSPBlock has:
* AXI-4 Stream input and output
* AXI-4 memory mapped status and control


<img src="images/fir_filter.png" style="width:800px;"/>

In [ ]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._

case object NumFilters extends CSRField {
    val name = "firQueueDepth"
}

case class FIRTap(filterIdx: Int, tapIdx: Int) extends CSRField {
    val name = s"firTap${filterIdx}_$tapIdx"
}

class FIRBlock(nFilters: Int, nTaps: Int)(implicit p: Parameters) extends
AXI4DspBlock with AXI4HasCSR with HasIPXactParameters {
    outer =>
    addStatus(NumFilters)
    
    for (i <- 0 until nFilters) {
        for (j <- 0 until nTaps) {
            addControl(FIRTap(i, j))
        }
    }
    makeCSRs()
    
    val streamNode = AXI4StreamIdentityNode()
    
    override def ipxactParameters = Map(
        "nFilters"   -> nFilters.toString,
        "nTaps"      -> nTaps.toString
    )

    def beatBytes: Int = 8
    def csrAddress = freechips.rocketchip.diplomacy.AddressSet(0x0, 0xffffL)
    def csrBase: Int = 0
    def csrSize: Int = 4096
    
    lazy val module = new LazyModuleImp(this) {
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)
        val mem = outer.mem.map { m => m.in(0) }
        
        require(in.params.n >= nFilters,
                s"""AXI-4 Stream port must be big enough for all 
                   |the filters (need $nFilters,, only have ${in.params.n})""".stripMargin)

        status(NumFilters) := nFilters.U
        
        
        val outs = (0 until nFilters).map(i => {
            val fir = Module(new MyManyDynamicElementVecFir(nTaps))
            
            fir.io.in := in.bits.data((i+1)*8, i*8)
            fir.io.valid := in.valid && out.ready
            
            for (j <- 0 until nTaps) {
                fir.io.consts(j) := control(FIRTap(i, j))
            }
            
            fir.io.out
        })
        
        
        val output = if (outs.length == 1) {
            outs(0)
        } else {
            outs.reduce((x: UInt, y: UInt) => Cat(y, x))
        }
            
        out.bits.data := output
        in.ready  := out.ready
        out.valid := in.valid
    }
}

---
# Invoking the Generator

The following code invokes the generator.
It will produce a file called `BlindModule.fir` which contains the firrtl generated by our code.
`BlindModule` is a wrapper for our `DspBlock`- it is necessary because of how [diplomacy](https://carrv.github.io/papers/cook-diplomacy-carrv2017.pdf) in rocket works.
You'll notice that `BlindModule` instantiates an `FIRBlock`.

The line
```scala
AXI4StreamBundleParameters(n = 8)
```
sets the AXI-4 streams' data widths to 8 bytes.

In [ ]:
object FIRGenerator extends ipxact.IPXactGeneratorApp with App {
  override val verilogFilename: String = "BlindModule.v"
  override val ipxactDir: String = "./"
  implicit val p: Parameters = Parameters.root((new freechips.rocketchip.coreplex.BaseCoreplexConfig).toInstance)

    val blindNodes = DspBlockBlindNodes.apply(
      AXI4StreamBundleParameters(n = 8),
      () => AXI4MasterNode(Seq(AXI4MasterPortParameters(Seq(AXI4MasterParameters("fir"))))))
    
    val dut = () => {
        val lazyMod = LazyModule(DspBlock.blindWrapper(() => new FIRBlock(4, 8), blindNodes))
        val m = lazyMod.module
        IPXactComponents._ipxactComponents += DspIPXact.makeDspBlockComponent(lazyMod.internal)
        m
    }
    
    chisel3.Driver.dumpFirrtl(chisel3.Driver.elaborate(dut), None)
    generateIPXact(IPXactComponents.ipxactComponents())
}
FIRGenerator.main(Array[String]())

---
# IPXact
Look in the file browser for a file called `craft_BlindModule_edu.berkeley.cs_1.0.xml`.
This is the ipxact file.
It contains information about:
* Port mappings
* Interfaces
* Memory maps
* Generator parameters

You'll notice that the parameters from the scala code
```scala
override def ipxactParameters = Map(
    "nFilters"   -> nFilters.toString,
    "nTaps"      -> nTaps.toString
)
```
appears in the IPXact output.
This gives verification generators the information they need to generate appropriate test vectors for the given instance.